# C1 : รวมการใช้งานแพ็กเกจ PyTorch ที่เกี่ยวข้อง

สมุดบันทึกเสริมนี้รวบรวมการใช้แพ็กเกจ PyTorch สนับสนุนการคำนวณในหนังสือ ซึ่งต้องติดตั้งก่อนโดยคำสั่ง 

```python
pip install torch 
```

รันเซลล์ด้านล่างนี้เพื่อนำเข้าและตรวจสอบเวอร์ชันของ torch 

In [1]:
import torch  
torch.__version__

'2.9.1+cpu'

## C1.1 การหาอนุพันธ์อัตโนมัติ 

ในภาคผนวก C ได้อธิบายวิธีการหาอนุพันธ์อัตโนมัติโดยใช้ Drake เป็นตัวอย่าง ในเอกสารนี้แสดงการใช้ torch 
จุดที่แตกต่างลำดับแรกคือ เมธอดต่างๆ ที่เคยใช้ numpy เช่น np.array(), np.sin() 
จุะต้องเรียกฟังก์ชันที่สนับสนุนการคำนวณของ torch แทน เช่น torch.tensor(), torch.sin() 

หลักการของการหาอนุพันธ์อัตโนมัติโดย torch คือเมื่อเรากำหนดอาร์กิวเมนต์ 
requires_grad=True PyTorch จะสร้างกราฟการคำนวณ (computational graph) เพื่อคำนวณอนุพันธ์โดยอัตโนมัติผ่านการแพร่กระจายย้อนหลัง (back propagation) 

***

**ตัวอย่าง C1.1** ทดสอบกับฟังก์ชันพื้นฐาน $y = f_1(x) = xsin(x)$

In [2]:
def f1(x):
    return x*torch.sin(x)

เมื่อกำหนดให้ $x = \frac{\pi}{6}$ เอาต์พุตของฟังก์ชันจะได้เป็นดังนี้

In [3]:
x1 = torch.tensor(torch.pi/6,requires_grad=True)
y = f1(x1)
print(y)

tensor(0.2618, grad_fn=<MulBackward0>)


โดยจะเห็นว่ามีส่วนเพิ่มเติม grad_fn 

จากการอนุพัทธ์เชิงสัญลักษณ์ อนุพันธ์ของฟังก์ชัน f1(x) มีค่าเท่ากับ $f_1'(x) = xcos(x) + sin(x)$ จากการคำนวณด้ายมือได้เท่ากับ

In [4]:
dy_a = x1*torch.cos(x1) + torch.sin(x1)
print("dy (คำนวณด้วยมือ) = {}".format(dy_a))

dy (คำนวณด้วยมือ) = 0.9534498453140259


ค่าที่ได้จากการคำนวณอนุพันธ์อัตโนมัติโดย torch คือ

In [5]:
y.backward()

In [6]:
dy = x1.grad
print("dy (จาก torch.autograd) = {}".format(dy))

dy (จาก torch.autograd) = 0.9534498453140259


In [7]:
print("ค่าแตกต่าง = {}".format(torch.linalg.norm(dy - dy_a)))

ค่าแตกต่าง = 0.0


***

**ตัวอย่าง C1.2** กรณีที่ฟังก์ชันมีอินพุตเป็นเวกเตอร์และเอาต์พุตเป็นสเกลาร์เช่น $f_2(x) = x_1^2 + 2x_2^2$ 

In [8]:
def f2(x):
    return x[0]**2 + 2*x[1]**2

อนุพันธ์จะเป็นเวกเตอร์
$$
f_2'(x) = 
\left[\begin{array}{c} 
\frac{\partial f_2(x)}{\partial x_1} \\
\frac{\partial f_2(x)}{\partial x_2} 
\end{array}\right] = 
\left[\begin{array}{c} 
2x_1 \\
4x_2
\end{array}\right] \tag{C1.1}
$$

เรียกว่าเกรเดียนต์ สมมุติว่ากำหนด $x_2 = [1.0, -1.0]$ ได้อนุพันธ์เป็นดังนี้ (ลองเปรียบเทียบกับการคำนวณด้วยมือจะเห็นว่าเท่ากัน)

In [9]:
x2 = torch.tensor([1.0,-1.0],requires_grad=True)
y2 = f2(x2)
print(y2)

tensor(3., grad_fn=<AddBackward0>)


In [10]:
y2.backward()

In [11]:
grad_fx2 = x2.grad
print(grad_fx2)

tensor([ 2., -4.])


***

**ตัวอย่าง C1.3** ระบบพลวัตที่เป็นเงื่อนไขบังคับของการควบคุมเหมาะที่สุดเช่น $x_{k+1} = f(x_k,u_k)$ 
หรือในกรณีทั่วไปคือ $y = f(x,u)$ อยู่ในรูปหลายอินพุตหลายเอาต์พุต เพื่อความง่ายพิจารณากรณีฟังก์ชันไม่เป็นเชิงเส้นที่ไม่มีอินพุต $u$

$$
\left[\begin{array}{c}
y_1 \\
y_2
\end{array}\right] =
\left[\begin{array}{c}
x_1^3  + 2x_2 \\
x_1cos(x_2)
\end{array}\right] \tag{C1.2}
$$

อนุพันธ์อยู่ในรูปเมทริกซ์เรียกว่าจาโคเบียน
$$
\left[\begin{array}{cc}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} \\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} \\
\end{array}\right] = 
\left[\begin{array}{cc}
3x_1^2 & 2 \\
cos(x_2) & -x_1sin(x_2) \\
\end{array}\right] \tag{C1.3}
$$

ในกรณีนี้ต้องใช้ torch.autograd.functional.jacobian ในการคำนวณจาโคเบียน
 

In [12]:
from torch.autograd.functional import jacobian

In [13]:
def f3(x):
    y = torch.zeros(2)
    y[0] = x[0]**3  + 2*x[1]
    y[1] = x[0]*torch.cos(x[1])
    return y

คำนวณอนุพันธ์ที่อินพุต $x = [1, \pi /2]$

In [14]:
x3 = torch.tensor([1.0, torch.pi/2],requires_grad=True)
J = jacobian(f3,x3)
print(J)

tensor([[ 3.0000e+00,  2.0000e+00],
        [-4.3711e-08, -1.0000e+00]])


เปรียบเทียบกับการคำนวณ C.3 ด้วยมือ

In [15]:
J_a = torch.tensor([[3*x3[0]**2, 2.0],[torch.cos(x3[1]),-x3[0]*torch.sin(x3[1])]])
print(J_a)

tensor([[ 3.0000e+00,  2.0000e+00],
        [-4.3711e-08, -1.0000e+00]])


/tmp/ipykernel_142/279788899.py:1: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  J_a = torch.tensor([[3*x3[0]**2, 2.0],[torch.cos(x3[1]),-x3[0]*torch.sin(x3[1])]])


จากการทดสอบเบื้องต้นพบว่าฟังก์ชันคำนวณอนุพันธ์อัตโนมัติของ PyTorch มีความยืดหยุ่นกว่า Drake 
ตัวอย่างเช่นในกรณีฟังก์ชันอินพุตเดียวหลายเอาต์พุต เมื่อใช้ Drake จะต้องแก้ไขโค้ดใน forwarddiff.py 
ให้รองรับ แต่หากใช้ torch สามารถใช้ torch.autograd.functional.jacobian ได้โดยไม่มีข้อผิดพลาด

***
    
**ตัวอย่าง C1.4** พิจารณาฟังก์ชันหนึ่งอินพุตสองเอาต์พุต f4() ดังนี้

In [16]:
def f4(u):
    y = torch.zeros(2)
    y[0] = torch.sin(u)
    y[1] = torch.cos(u)
    return y

สามารถคำนวณอนุพันธ์สำหรับ $u = 1.0$ ได้โดยใช้ torch.autograd.functional.jacobian

In [17]:
u4 = torch.tensor(1.0,requires_grad=True)
J4 = jacobian(f4,u4)
print(J4)

tensor([ 0.5403, -0.8415])


ในกรณีที่ฟังก์ชันมีการคำนวณซับซ้อนขึ้น เช่นมีการหาคำตอบพีชคณิตเชิงเส้น 
เรายังสามารถที่จะคำนวณจาโคเบียนของฟังก์ชันได้ เพียงแต่ต้องระมัดระวังในการเขียนโปรแกรมมากขึ้น 
เช่นใช้ฟังก์ชันของ torch แทนการใช้ numpy ทั้งหมดเพื่อรองรับการคำนวณอนุพันธ์อัตโนมัติ
นอกจากนั้นยังมีบางจุดที่ผู้เขียนทดสอบแล้วไม่สามารถคำนวณค่าอนุพันธ์ได้อย่างถูกต้อง
แม้ว่าโค้ดจะใช้งานได้สำหรับการคำนวณปกติ 

***

**ตัวอย่าง C1.5** สมมุติว่าโมเดลพลวัตประกอบด้วยอินพุตเวกเตอร์ $x = [x_1, x_2]$  
และสเกลาร์ $u$  สร้างเมทริกซ์ $A$ และเวกเตอร์ $B$ ดังนี้
$$
A = \left[\begin{array}{cc}
3x_1^2 & x_1x_2 \\
2x_1x_2 & -4x_2^2
\end{array}\right] \tag{C1.4}
$$
$$
B = \left[\begin{array}{c}
0.0 \\
u
\end{array}\right] \tag{C1.5}
$$
คำนวณเอาต์พุต $y$ ได้จาก
$$
y = A^{-1}B \tag{C1.6}
$$
ต้องการคำนวณจาโคเบียนของ $y$ เทียบกับ $x$ และ $u$ 

**หมายเหตุ :** ฟังก์ชันคำนวณจาโคเบียนของ torch นอกจาก jacobian แล้ว 
หากเราต้องการระบุชนิดของการคำนวณอนุพันธ์ว่าเป็นแบบข้างหน้าหรือย้อนหลัง สามารถเลือกใช้ 
jacfwd หรือ jacrev ขึ้นกับว่าฟังก์ชันที่ต้องการหาอนุพันธ์มีจำนวนอินพุตมากหรือน้อยกว่าเอาต์พุต 
แต่ในโมเดลขนาดเล็กจะไม่เห็นความแตกต่างด้านสมรรถนะเท่าไรนัก ในตัวอย่างนี้จะทดลองใช้ jacfwd
นำเข้าได้ดังนี้ (ได้นำเข้า jacrev ไว้ด้วยเผื่อผู้อ่านต้องการทดสอบ)

In [18]:
from torch.func import jacfwd, jacrev

ในครั้งแรกจะทดลองเขียนฟังก์ชัน f5() ในแบบที่คุ้นเคย ซึ่งจะพบว่าคำนวณอนุพันธ์ได้คำตอบเป็นศูนย์

In [19]:
# problematic function definition
def f5(x,u):
    A = torch.tensor([[3*x[0]**2,x[0]*x[1]],[2*x[0]*x[1],-4*x[1]**2]])
    B = torch.tensor([0.0,u])
    y = torch.linalg.solve(A,B)
    return y

ในการสร้างอินพุตจะต้องมีอาร์กิวเมนต์ requires_grid = True 

ข้อดีในการใช้ฟังก์ชัน jacfwd() เมื่อเปรียบเทียบกับ Drake คือมีอาร์กิวเมนต์ argnums 
ที่สามารถกำหนดอินพุตที่ต้องการเทียบอนุพันธ์ได้ เช่น argnums=0 (ค่าโดยปริยาย) 
จะกำหนดให้คำนวณจาโคเบียนเทียบกับอินพุตแรกคือ $x$ 

In [20]:
x5 = torch.tensor([0.5, 0.2],requires_grad=True)
u5 = torch.tensor([10.0],requires_grad=True)
jacfwd(f5,argnums=0)(x5,u5)

tensor([[0., 0.],
        [0., 0.]])

ค่าอนุพันธ์เป็นศูนย์หมดนี้คือคำตอบที่ไม่ถูกต้อง จากการตรวจสอบและแก้ไขโค้ด 
ผู้เขียนพบว่าการเขียนฟังก์ชันเพื่อให้คำนวณอนุพันธ์ได้ถูกต้องจะต้องนิยามเมทริกซ์ $A,B$ ขึ้นมาก่อน 
โดยมีค่าสมาชิกเป็นศูนย์ จากนั้นจึงคำนวณเพื่อแทนค่าแต่ละสมาชิกตาม (C.4)

In [21]:
def f5(x,u):
    A = torch.zeros(2,2)
    B = torch.zeros(2,1)
    A[0,0] = 3*x[0]**2
    A[0,1] = x[0]*x[1]
    A[1,0] = 2*x[0]*x[1]
    A[1,1] = -4*x[1]**2
    B[1,0] = u
    y = torch.linalg.solve(A,B)
    return y

เมื่อคำนวณอนุพันธ์จะได้ค่าที่ถูกต้อง (ตรวจสอบกับแพ็กเกจ JAX ได้ค่าเท่ากัน)

In [22]:
x5 = torch.tensor([0.5, 0.2],requires_grad=True)
u5 = torch.tensor([10.0],requires_grad=True)
jacfwd(f5,argnums=0)(x5,u5)

tensor([[[-1.4286e+01, -3.5714e+01]],

        [[-1.2772e-06,  5.3571e+02]]], grad_fn=<ViewBackward0>)

หากต้องการหาอนุพันธ์เทียบกับ $u$ ให้ใส่อาร์กิวเมนต์ argnums=1

In [23]:
jacfwd(f5,argnums=1)(x5,u5)

tensor([[[ 0.7143]],

        [[-5.3571]]], grad_fn=<ViewBackward0>)

***

**ตัวอย่าง C1.6** พิจารณาฟังก์ชันที่รับอินพุต $x$ เป็นเวกเตอร์ 2 สมาชิก $u$ เป็นสเกลาร์ สร้างเมทริกซ์ 
$M$ ขนาด 2x2 จาก $x$ เวกเตอร์ $\tau$ จาก $u$ และคืนค่า y = torch.linalg.solve($M,\tau$) 

**หมายเหตุ :** โมเดลนี้เป็นพื้นฐานของพลวัตที่ใช้ในหนังสือ ทีมีการคำนวณสมการเชิงเส้น
$$
My = \tau \tag{C1.7}
$$

ตัวอย่างเช่นหุ่นยนต์กายกรรม

สร้างฟังก์ชัน f6() ดังนี้ โดยใช้โครงสร้างเดียวกับตัวอย่าง C1.5

In [24]:
def f6(x,u):
    s1 = torch.sin(x[0])
    c1 = torch.cos(x[0])
    s12 = torch.sin(x[0]+x[1])    
    c12 = torch.cos(x[0]+x[1])
    m11 = s1 + c12
    m12 = c1 + s12
    m22 = 2.0
    M = torch.zeros(2,2)
    M[0,0] = m11
    M[0,1] = m12
    M[1,0] = m12
    M[1,1] = m22
    tau = torch.zeros(2,1)
    tau[1,0] = u
    y = torch.linalg.solve(M,tau)
    return y

คำนวณเอาต์พุตของโมเดล f6() เมื่อกำหนดอินพุต $x = [-\pi/4, \; \pi/2]$ และ $u = 1.0$

In [25]:
x6 = torch.tensor([-torch.pi/4, torch.pi/2])
u6 = torch.tensor(1.0)
f6(x6,u6)

tensor([[0.7071],
        [0.0000]])

คำนวณอนุพันธ์เทียบ $x$

In [26]:
x6 = torch.tensor([-torch.pi/4, torch.pi/2],requires_grad=True)
u6 = torch.tensor(1.0,requires_grad=True)
jacfwd(f6,argnums=0)(x6,u6)

tensor([[[-0.7071, -0.8536]],

        [[ 0.0000,  0.3536]]], grad_fn=<ViewBackward0>)

คำนวณอนุพันธ์เทียบ $u$

In [27]:
jacfwd(f6,argnums=1)(x6,u6)

tensor([[0.7071],
        [0.0000]], grad_fn=<SqueezeBackward1>)

### C1.1.1 การประมาณค่าโมเดลเชิงเส้น

ในเนื้อหาที่ตามรอยรายวิชา [2] นิยมใช้การหาอนุพันธ์อัตโนมัติเพื่อประมาณค่าโมเดลเชิงเส้นแบบดีสครีตจากพลวัตไม่เป็นเชิงเส้น 
โดยอาศัยฟังก์ชันคำนวณอนุพันธ์โดยวิธีรุงเง คุตตา ในหัวข้อนี้จะทดสอบวิธีการดังกล่าวโดยเปรียบเทียบกับการประมาณค่าบนกระดาษ

**ตัวอย่าง C1.7** พลวัตของคาร์ทโพล [3]

เชื่อว่าผู้ศึกษาระบบควบคุมคงเคยเห็นหรือคุ้นเคยกับคาร์ทโพลในรูปที่ C1.1 หรืออาจรู้จักในชื่อ "ลูกตุ้มหัวกลับบนรถเลื่อน" 
วัตถุประสงค์การควบคุมคือรักษาดุลให้ลูกตุ้มอยู่ในแนวดิ่ง เรียกว่าเป็นจุดสมดุลที่ไม่เสถียร (unstable equilibrium) 
โดยอาศัยแรงในแนวนอนที่กระทำกับรถเข็นเพียงอย่างเดียว ลักษณะเหมือนกับเวลาเรารักษาดุลของไม้กวาดบนมือ 
เพียงแต่ปัญหาถูกจำกัดอยู่ในระนาบ 2 มิติ รายละเอียดเพิ่มเติมของพลวัตของคาร์ทโพลสามารถศึกษาได้จาก
[Section 3.2 of Underactuated Robotics](https://underactuated.csail.mit.edu/acrobot.html#cart_pole) 
<div align="center">
<img src="https://raw.githubusercontent.com/dewdotninja/sharing-github/master/cart_pole.png" width=500 />
</div>
<div align="center">รูปที่ C1.1 ระบบคาร์ทโพลในระนาบ 2 มิติ</div>

นำเข้าแพ็กเกจที่ต้องการใช้งาน

In [28]:
import numpy as np 
from numpy.linalg import norm
import matplotlib.pyplot as plt 
import control as ctl
import torch
from torch.func import jacfwd, jacrev

เพื่อทำให้ปัญหาง่ายขึ้น แทนค่าพารามิเตอร์ดังนี้

- มวลของรถเข็น $m_c=1$,
- มวลของลูกตุ้ม $m_p=1$,
- ความยาวของเสาลูกตุ้ม $l=1$,
- แรงโน้มถ่วงโลก $g=9.81$.

สถานะของคาร์ทโพลนิยามได้เป็น $\mathbf{x} = [x, \theta, \dot{x}, \dot{\theta}]^T$ 
และแรงที่กระทำกับรถเข็นคืออินพุตควบคุม $\mathbf{u} = [f_x]$

ใช้ [สมการ (16) และ (17)](https://underactuated.csail.mit.edu/acrobot.html#cart_pole) จาก [3]
ที่บรรยายความเร่งเชิงเส้นและเชิงมุมของคาร์ทโพล

$$
\ddot{x} = \frac{1}{m_c + m_p \sin^2\theta}
[ f_x+m_p \sin\theta (l \dot\theta^2 + g\cos\theta)] \tag{C1.8}
$$

$$
\ddot{\theta} = \frac{1}{l(m_c + m_p \sin^2\theta)}
[ -f_x \cos\theta - m_p l \dot\theta^2 \cos\theta \sin\theta - (m_c + m_p) g \sin\theta] \tag{C.9}
$$

อิมพลิเมนต์เวกเตอร์
$\dot{\mathbf{x}} = [\dot{x}, \dot{\theta}, \ddot{x}, \ddot{\theta}]^T$ 
เป็น $\dot{\mathbf{x}} = {\bf f}(\mathbf{x}, \mathbf{u})$. 
ในฟังก์ชัน ${\bf f}$ ด้านล่างเพื่อคืนค่า $\dot{\mathbf{x}}$


In [29]:
# modified to use torch Autograd
def cartpole(x, u):
    c = torch.cos(x[1])  # cos(theta)
    s = torch.sin(x[1])  # sin(theta)
    # parameters
    m_c = 1.0  # cart mass
    m_p = 1.0  # pendulum mass
    l = 1.0  # pole length
    g = 9.81  # gravity

    y = torch.zeros(4,1)
    y[0,0] = x[2]  # x_dot is the 3rd element of state vector
    y[1,0] = x[3]  # theta_dot is the 4th element of state vector
    y[2,0] = (1/(m_c + m_p*s**2))*(u + m_p*s*(l*x[3]**2+g*c))  # from (16) of [3]
    y[3,0] = (1/(l*(m_c + m_p*s**2)))*(-u*c - m_p*l*x[3]**2*c*s - (m_c+m_p)*g*s)  # from (17) of [3]

    return y

ต้องการแปลงเป็นเชิงเส้นรอบจุดสมดุลที่ไม่เสถียร 
พิจารณาสถานะสมดุลที่ไม่เสถียร 
$$\mathbf{x}^* = [0, \pi, 0, 0]^T \tag{C1.10}
$$
โดยมีอินพุตควบคุม ณ จุดสมดุลนี้เป็น
$$
\mathbf{u}^* = [0] \tag{C1.11}
$$

ใช้วิธีการที่กล่าวถึงในหนังสือ
[[3]](https://underactuated.csail.mit.edu/acrobot.html#linearizing_manip)
ในการอนุพัทธ์โมเดลเชิงเส้นในรูป
$$\dot{\bar{\mathbf{x}}} = A_{\text{lin}} \mathbf{\bar{x}} + B_{\text{lin}} \mathbf{\bar{u}},$$
where $\mathbf{\bar{x}} = \mathbf{x}-\mathbf{x}^*$ and $\mathbf{\bar{u}} = \mathbf{u} -\mathbf{u}^*$.

รายละเอียดแสดงในรูปที่ C1.2                          
<div align="center">
<img src="https://raw.githubusercontent.com/dewdotninja/sharing-github/master/cardpole_linearize.png" width=600 />
</div>
<div align="center">
รูปที่ C1.2 รายละเอียดการแปลงคาร์ทโพลเป็นระบบเชิงเส้น
</div>

สร้างเมทริกซ์ $A_{c1}$ และเวกเตอร์ $B_{c1}$
ตามรายละเอียดในรูปที่ C1.2 

In [30]:
g = 9.81  # gravity
A_c1 = np.array(
    [
        [0, 0, 1, 0],  
        [0, 0, 0, 1],  
        [0, g, 0, 0], 
        [0, 2*g, 0, 0],  
    ]
)
B_c1 = np.array(
    [
        [0],
        [0],
        [1],
        [1],
    ]  
)


แปลงเป็นระบบดีสครีต $A_{d1}, B_{d1}$ โดยใช้แพ็กเกจ Python Control 
เลือกคาบเวลา $h = 0.01$ วินาที

In [31]:
h = 0.01 # period (sec)
C_c1 = np.array([1,0,0,0])
sys_c = ctl.ss(A_c1,B_c1,C_c1,0)
sys_d = ctl.sample_system(sys_c,h)
A_d1,B_d1,C_d1,D_d1 = ctl.ssdata(sys_d)

สำหรับวิธีการประมาณค่าเชิงเส้นระบบดีสครีตโดยใช้การหาอนุพันธ์อัตโนมัติ เริ่มจากการนิยามฟังก์ชัน 
สำหรับการหาปริพันธ์โดยวิธีรุงเงอ-คุตตา 

In [32]:
def cartpole_rk4(x,u):
    #RK4 integration with zero-order hold on u
    #x_1 = x
    f1 = cartpole(x, u)
    
    #x_a = x.reshape(4,1)
    x_2 = x + 0.5*h*f1
    f2 = cartpole(x_2, u)
    
    x_3 = x + 0.5*h*f2
    f3 = cartpole(x_3, u)

    x_4 = x + 0.5*h*f3
    f4 = cartpole(x_4, u)

    ft = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    
    return ft

สร้างฟังก์ชันหาอนุพันธ์เทียบ $x$ และ $u$ 

In [33]:
def cartpole_dfdx(x,u):
    return jacfwd(cartpole_rk4,argnums=0)(x,u)

def cartpole_dfdu(x,u):
    return jacfwd(cartpole_rk4,argnums=1)(x,u)    

กำหนดจุดสมดุลตาม (C1.10), (C1.11)

In [34]:
x_star = torch.tensor([0, torch.pi, 0, 0],requires_grad=True).reshape(4,1)
u_star = torch.tensor(0.0,requires_grad=True)

ใช้่การหาอนุพันธ์อัตโนมัติเพื่อประมาณค่าโมเดลเชิงเส้น

In [35]:
Ad2 = cartpole_dfdx(x_star,u_star)
A_d2 = torch.squeeze(Ad2.detach()).numpy()
A_d2

array([[1.0000000e+00, 4.0879013e-04, 9.9999998e-03, 1.2262501e-06],
       [0.0000000e+00, 1.0008175e+00, 0.0000000e+00, 1.0002454e-02],
       [0.0000000e+00, 9.8124065e-02, 1.0000000e+00, 4.0879013e-04],
       [0.0000000e+00, 1.9624813e-01, 0.0000000e+00, 1.0008175e+00]],
      dtype=float32)

In [36]:
Bd2 = cartpole_dfdu(x_star,u_star)
B_d2 = Bd2.detach().numpy()
B_d2

array([[4.1668711e-05],
       [4.1670755e-05],
       [1.0001225e-02],
       [1.0002454e-02]], dtype=float32)

เขียนฟังก์ชันเพื่อเปรียบเทียบสถานะ $x_{k+1} = Ax_k + Bu_k$
จากการโมเดลบนกระดาษกับการคำนวณอนุพันธ์อัตโนมัติ 
โดยให้ $x_k, u_k$ เป็นค่าสุ่ม 

In [37]:
x_star_np = x_star.detach().numpy()
u_star_np = u_star.detach().numpy()

In [38]:
def cartpole_linear(A,B,x,u):
    x_bar = x - x_star_np
    u_bar = u - u_star_np
    return A.dot(x_bar) + B.dot(u_bar) 

def linearized_difference(x,u):
    return norm(cartpole_linear(A_d1,B_d1,x,u) - cartpole_linear(A_d2,B_d2,x,u))
    

ทดลองรันเซลล์นี้หลายครั้งเพื่อดูผลการเปรียบเทียบ ค่าความแตกต่างจะต้องมีค่าน้อย

In [39]:
x_test = np.random.rand(4,1)
u_test = np.random.rand()
linearized_difference(x_test,u_test)

np.float64(0.0005100591707641022)

### C1.1.2 คำนวณอนุพันธ์ผ่านฟังก์ชันรุงเงอ คุตตา

ในการหาโมเดลระบบดีสครีตของคาร์ทโพลในหัวข้อ C1.1.1 เราใช้การหาอนุพันธ์อัตโนมัติผ่านฟังก์ชันรุงเงอ 
คุตตา (RK4) ซึ่งในหนังสือนี้จะใช้วิธีการนี้อยู่ตลอด 
ดังนั้นในหัวข้อนี้เราจะตรวจสอบการคำนวณอนุพันธ์อัตโนมัติผ่าน RK4 
เปรียบเทียบผลกับการคำนวณโดยเขียนฟังก์ชันเอง 
โดยเมื่อเข้าใจแล้วเราสามารถเพิ่มการคำนวณอนุพันธ์ด้วยตัวเองในส่วนที่แพ็กเกจไม่สามารถทำได้ 

**ตัวอย่าง C1.8** ตัวอย่างนี้จะขยายจากโจทย์เดิมในตัวอย่าง C1.3 พิจารณาฟังก์ชันไม่เป็นเชิงเส้น 

$$
f(x) =
\left[\begin{array}{c}
f_1(x) \\
f_2(x)
\end{array}\right] =
\left[\begin{array}{c}
x_1^3  + 2x_2 \\
x_1cos(x_2)
\end{array}\right] \tag{C1.13}
$$

คำนวณจาโคเบียนได้เท่ากับ
$$
\left[\begin{array}{cc}
\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} \\
\frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} \\
\end{array}\right] = 
\left[\begin{array}{cc}
3x_1^2 & 2 \\
cos(x_2) & -x_1sin(x_2) \\
\end{array}\right] \tag{C1.14}
$$
 

In [40]:
import numpy as np
import torch
from torch.func import jacfwd, jacrev

In [41]:
def f(x):
    y = torch.zeros(2,1)
    y[0,0] = x[0]**3  + 2*x[1]
    y[1,0] = x[0]*torch.cos(x[1])
    return y

เขียนฟังก์ชัน rk4_f() เพื่อหาคำตอบโดยใช้การคำนวณปริพันธ์แบบ RK4 กำหนดค่าขั้น h = 0.05

In [42]:
h  = 0.05
def rk4_f(x):
    #RK4 integration 
    f1 = f(x)

    x_2 = x + 0.5*h*f1
    f2 = f(x_2)
    
    x_3 = x + 0.5*h*f2
    f3 = f(x_3)

    x_4 = x + 0.5*h*f3
    f4 = f(x_4)

    ft = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    
    return ft

คำนวณอนุพันธ์ที่อินพุต $x = [1, \pi /2]$ โดยใช้ jacfwd (หรือ jacrev) 

In [43]:
x = torch.tensor([1.0, torch.pi/2],requires_grad=True).reshape(2,1)
y_ad = torch.squeeze(jacfwd(rk4_f)(x).detach()).numpy()
print(y_ad)

[[ 1.1924909e+00  1.0582062e-01]
 [-2.2966973e-09  9.4667363e-01]]


ในการศึกษาการคำนวณอนุพันธ์อัตโนมัติผ่าน RK4 เราจะเปรียบเทียบกับการคำนวณอนุพันธ์ด้วยมือ 
ซึ่งไม่จำเป็นต้องใช้ torch ดังนั้นเพื่อความง่ายจะเขียนฟังก์ชัน f_np() สำหรับ (C1.13) โดยใช้ numpy แทน torch 
และฟังก์ชัน rk4_f_np() สำหรับ RK4 โดยเรียกฟังก์ชัน f_np() ในแต่ละครั้งของการหาปริพันธ์

In [44]:
# use numpy instead of torch for simplicity
def f_np(x):  
    f_1 = x[0]**3  + 2*x[1]
    f_2 = x[0]*np.cos(x[1])
    return np.array([f_1,f_2])

เขียนฟังก์ชัน jacobian_f() เพื่อคำนวณอนุพันธ์ด้วยตัวเองตาม (C1.14)

In [45]:
def jacobian_f(x):
    m_00 = 3*x[0]**2
    m_01 = 2
    m_10 = np.cos(x[1])
    m_11 = -x[0]*np.sin(x[1])
    M = np.array([[m_00, m_01],[m_10, m_11]])
    return M

โดยจากการตรวจสอบค่าของเอาต์พุตฟังก์ชันและอนุพันธ์ที่ได้จากการเรียกฟังก์ขัน f() 
ผ่าน jacfwd ในแต่ละครั้ง เราเขียนฟังก์ชัน jacobian_rk4_f() ได้ดังนี้

In [46]:
def jacobian_rk4_f(x):
    # manuallydo RK4 on derivatives
    # 1. you must start initial xd as np.eye(2)
    # 2. you must keep track of changes in x value and evaluate 
    # derivative at that point!
    
    xd_1 = np.eye(2) # initial xd
    f1 = f_np(x)  # function value
    df1 = jacobian_f(x) # df1 is 2x2 matrix

    x_2 = x + 0.5*h*f1 # update x for 2nd f() call
    xd_2 = xd_1 + 0.5*h*df1 # update Jacobian
    f2 = f_np(x_2)
    df2 = jacobian_f(x_2)

    x_3 = x + 0.5*h*f2
    xd_3 = xd_1 + 0.5*h*df2
    f3 = f_np(x_3)
    df3 = jacobian_f(x_3)

    x_4 = x + 0.5*h*f3

    xd_4 = xd_1 + 0.5*h*df3
    f4 = f_np(x_4)
    df4 = jacobian_f(x_4)

    dft = xd_1 + (h/6.0)*(df1 + 2*df2 + 2*df3 + df4)
    
    return dft 

อธิบายการคำนวณอนุพันธ์ผ่าน RK4 ในฟังก์ชัน jacobian_rk4_f() ได้ดังนี้ 
ในแต่ละครั้งของการเรียก f() 

1. คำนวณเอาต์พุตของฟังก์ชันเพื่อใช้ในการอัพเดตค่า x ในครั้งต่อไป 
2. คำนวณค่าอนุพันธ์ในแต่ละครั้งโดยใช้ค่า x ที่อัพเดตจากข้อ 1 โดยเป็นค่าเมทริกซ์จาโคเบียน 
สังเกตว่าอนุพันธ์เริ่มต้นที่ตำแหน่ง x คือ

$$
\left[\begin{array}{cc}
\frac{\partial x_1}{\partial x_1} & \frac{\partial x_2}{\partial x_1} \\
\frac{\partial x_2}{\partial x_1} & \frac{\partial x_2}{\partial x_2}
\end{array}\right] = 
\left[\begin{array}{cc}
1 & 0 \\
0 & 1
\end{array}\right] 
$$

ในขั้นตอนสุดท้าย เฉลี่ยค่าเมทริกซ์จาโคเบียนทั้ง 4 ครั้งตามสูตรของ RK4 ได้เป็นค่าเอาต์พุต
    
เมื่อคำนวณจาโคเบียนผ่าน RK4 ด้วยตัวเอง ได้ผลดังนี้

In [47]:
x_ = np.array([1.0, np.pi/2])
y_md = jacobian_rk4_f(x_)
y_md

array([[1.17869770e+00, 1.00000000e-01],
       [3.06161700e-18, 9.45464314e-01]])

จะเห็นว่าได้ผลใกล้เคียงกับการคำนวณอัตโนมัติ 
ในเซลล์ด้านล่างนี้เป็นการเปรียบเทียบโดยใช้อินพุต x เป็นค่าสุ่ม ทดลองรันเซลล์นี้หลายครั้ง 
แต่ละครั้งความแตกต่างต้องมีค่าน้อย

In [48]:
x_test = torch.rand(2,1,requires_grad=True)
y_ad = torch.squeeze(jacfwd(rk4_f)(x_test).detach()).numpy()
y_md = jacobian_rk4_f(np.squeeze(x_test.detach().numpy()))
np.linalg.norm(y_ad - y_md)

np.float64(0.005764410209389756)

### C1.1.3 การคำนวณเฮสเซียนอัตโนมัติ

torch มีฟังก์ชันสนับสนุนการคำนวณอนุพันธ์อันดับสูงเช่นเฮสเซียนเหมือนกับแพ็กเกจ JAX ซึ่งเป็นข้อได้เปรียบเหนือ Drake 
อย่างไรก็ตามการคำนวณเฮสเซียนอัตโนมัติมีความซับซ้อนและมักใช้เวลาในการคำนวณมาก โดยเฉพาะเมื่อมีตัวแปรจำนวนมาก 
ในส่วนนี้จะศึกษาการคำนวณเฮสเซียนอัตโนมัติ โดยเปรียบเทียบผลและเวลาการคำนวณระหว่าง torch กับ JAX

เนื่องจากจะมีความแตกต่างเล็กน้อยในการเขียนโค้ดเพื่อรองรับแพ็กเกจทั้งสอง ดังนั้นเพื่อความสะดวกในการรัน 
จะสร้างตัวแปรธงชื่อ ad_pkg โดยให้ค่าเป็น 0 หรือ 1 สำหรับแพ็กเกจ torch และ JAX ตามลำดับ

ในการทดสอบนี้จะใช้พลวัตของ cartpole เหมือนในหัวข้อ C1.1.1 

In [49]:
import numpy as np
import torch
from torch.func import jacfwd, hessian
import jax
import jax.numpy as jnp

In [50]:
ad_pkg = 0  # set to 0/1 for torch/jax

In [51]:
def cartpole(x, u):
    # parameters
    m_c = 1.0  # cart mass
    m_p = 1.0  # pendulum mass
    l = 1.0  # pole length
    g = 9.81  # gravity
    
    if ad_pkg == 0: # torch
        c = torch.cos(x[1])  # cos(theta)
        s = torch.sin(x[1])  # sin(theta)
        y = torch.zeros(4,1)
        y[0,0] = x[2]  # x_dot is the 3rd element of state vector
        y[1,0] = x[3]  # theta_dot is the 4th element of state vector
        y[2,0] = (1/(m_c + m_p*s**2))*(u + m_p*s*(l*x[3]**2+g*c))  # from (16) of [3]
        y[3,0] = (1/(l*(m_c + m_p*s**2)))*(-u*c - m_p*l*x[3]**2*c*s - (m_c+m_p)*g*s)  # from (17) of [3]
    else: # jax
        c = jnp.cos(x[1])
        s = jnp.sin(x[1])
        y = jnp.zeros((4,1))
        f1 = x[2]  # x_dot is the 3rd element of state vector
        f2 = x[3]  # theta_dot is the 4th element of state vector
        f3 = (1/(m_c + m_p*s**2))*(u + m_p*s*(l*x[3]**2+g*c))   
        f4 = (1/(l*(m_c + m_p*s**2)))*(-u*c - m_p*l*x[3]**2*c*s - (m_c+m_p)*g*s)  
        y = jnp.array([f1, f2, f3, f4]).reshape(4,1)
    return y

In [52]:
h = 0.05 # time step

def cartpole_rk4(x,u):
    #RK4 integration with zero-order hold on u
    f1 = cartpole(x, u)
    
    x_2 = x + 0.5*h*f1
    f2 = cartpole(x_2, u)
    
    x_3 = x + 0.5*h*f2
    f3 = cartpole(x_3, u)

    x_4 = x + 0.5*h*f3
    f4 = cartpole(x_4, u)

    ft = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)    
    return ft

ในการคำนวณเฮสเซียนสำหรับคาร์ทโพลจะได้เอาต์พุตเป็นเทนเซอร์ 
เราจะต้องแปลงให้เป็นเมทริกซ์ตามรายละเอียดที่อธิบายในภาคผนวกอื่นของหนังสือ 
เขียนฟังก์ชันการแปลงดังนี้

In [53]:
# convert [m,n,p] tensor to [m*n,p] matrix
def tensor2mat(Mat3d):
    dim3 = Mat3d.shape[2]
    M = Mat3d[:,:,0]
    for i in range(dim3-1):
        if ad_pkg == 0: # torch
            M = torch.vstack([M,Mat3d[:,:,i+1]])
        else: # jax
            M = jnp.vstack((M,Mat3d[:,:,i+1]))
    return M

นอกจากนั้นเราต้องการแปลงเมทริกซ์เป็นเวกเตอร์คอลัมน์ในฟังก์ชัน dAdu() ด้านล่าง 
เขียนเป็นฟังก์ชันสนับสนุนเพิ่มดังนี้

In [54]:
# vertical stack a matrix to column vector
def mat2colvec(mat):
    dim0 = mat.shape[0]
    dim1 = mat.shape[1]
    colvec = mat[:,[0]]
    for i in range(1,dim1):
        if ad_pkg==0: 
            colvec = torch.vstack([colvec,mat[:,[i]]])
        else:
            colvec = jnp.vstack((colvec,mat[:,[i]]))
    return colvec

เพื่อความสมบูรณ์จะรวมโค้ดสำหรับคำนวณเกรเดียนต์และเฮสเซียนที่ใช้ในบทที่ 5 
โดยใช้ตัวแปร ad_pkg เป็นตัวเลือกระหว่าง torch กับ JAX
และทดลองจับเวลาการคำนวณเปรียบเทียบกัน

In [55]:
def dfdx(x,u):
    if ad_pkg==0:
        y = torch.squeeze(jacfwd(cartpole_rk4,argnums=0)(x,u))
    else:
        y = jnp.squeeze(jax.jacfwd(cartpole_rk4,argnums=0)(x,u))
    return y

def dfdu(x,u):
    if ad_pkg==0:
        y = jacfwd(cartpole_rk4,argnums=1)(x,u)
    else:
        y = jax.jacfwd(cartpole_rk4,argnums=1)(x,u)
    return y

def dAdx(x,u):
    if ad_pkg==0:
        y = hessian(cartpole_rk4)(x,u) # argnums=0 is default
    else:
        y = jax.hessian(cartpole_rk4)(x,u)
    return tensor2mat(y)[:,0,:]

def dBdx(x,u):
    if ad_pkg==0:
        y = torch.squeeze(jacfwd(dfdu)(x,u))
    else:    
        y = jnp.squeeze(jax.jacfwd(dfdu)(x,u))
    return y

def dAdu(x,u):
    if ad_pkg==0:
        y = jacfwd(dfdx,argnums=1)(x,u)
        yr = mat2colvec(y)
    else:    
        y = jax.jacfwd(dfdx,argnums=1)(x,u)
        yr = mat2colvec(y)
    return yr

def dBdu(x,u):
    if ad_pkg==0:
        y = jacfwd(dfdu,argnums=1)(x,u)
    else:    
        y = jax.jacfwd(dfdu,argnums=1)(x,u)
    return y


เริ่มต้นจากการใช้ torch คำนวณฟังก์ชันที่เป็นการหาอนุพันธ์อันดับสอง คือ 
dAdx(), dBdx(), dAdu(), dBdu() โดยมีการจับเวลาที่ใช้ในการคำนวณ ใช้โค้ดดังนี้

```python
import time
start_time = time.perf_counter()
# put code block to measure execution time here
end_time = time.perf_counter()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.6f} seconds")
```

In [56]:
ad_pkg = 0  # select torch AD
x_star = torch.tensor([0.0, torch.pi, 0.0, 0.0],requires_grad=True).reshape(4,1)
u_star = torch.tensor(1.0, requires_grad=True)

In [57]:
import time

In [58]:
# dAdx (torch)
start_time = time.perf_counter()
Ax_torch = torch.squeeze(dAdx(x_star,u_star)).detach().numpy()
end_time = time.perf_counter()
torch_dadx_time = end_time - start_time
print(f"Elapsed time (torch : dAdx) : {torch_dadx_time:.6f} seconds")
Ax_torch

Elapsed time (torch : dAdx) : 0.049429 seconds


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -2.14138301e-03,  0.00000000e+00,
        -6.37206031e-05],
       [ 0.00000000e+00, -3.19725974e-03,  0.00000000e+00,
        -7.95387823e-05],
       [ 0.00000000e+00, -1.05343044e-01,  0.00000000e+00,
        -4.30992106e-03],
       [ 0.00000000e+00, -1.57050446e-01,  0.00000000e+00,
        -5.38383238e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
      

In [59]:
# dBdx (torch)
start_time = time.perf_counter()
Bx_torch = dBdx(x_star,u_star).detach().numpy()
end_time = time.perf_counter()
torch_dbdx_time = end_time - start_time
print(f"Elapsed time (torch : dBdx) : {torch_dbdx_time:.6f} seconds")
Bx_torch

Elapsed time (torch : dBdx) : 0.069097 seconds


array([[ 0.0000000e+00, -1.3262699e-06,  0.0000000e+00, -4.5935426e-08],
       [ 0.0000000e+00, -1.5931810e-06,  0.0000000e+00, -5.2526463e-08],
       [ 0.0000000e+00, -1.1780268e-04,  0.0000000e+00, -4.4945300e-06],
       [ 0.0000000e+00, -1.4988930e-04,  0.0000000e+00, -5.2902114e-06]],
      dtype=float32)

In [60]:
# dAdu (torch)
start_time = time.perf_counter()
Au_torch = dAdu(x_star,u_star).detach().numpy()
end_time = time.perf_counter()
torch_dadu_time = end_time - start_time
print(f"Elapsed time (torch : dAdu) : {torch_dadu_time:.6f} seconds")
Au_torch

Elapsed time (torch : dAdu) : 0.071452 seconds


array([[ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [-1.3262699e-06],
       [-1.5931810e-06],
       [-1.1780267e-04],
       [-1.4988930e-04],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [-4.5935426e-08],
       [-5.2526467e-08],
       [-4.4945300e-06],
       [-5.2902114e-06]], dtype=float32)

In [61]:
# dBdu (torch)
start_time = time.perf_counter()
Bu_torch = dBdu(x_star,u_star).detach().numpy()
end_time = time.perf_counter()
torch_dbdu_time = end_time - start_time
print(f"Elapsed time (torch : dBdu) : {torch_dbdu_time:.6f} seconds")
Bu_torch

Elapsed time (torch : dBdu) : 0.069050 seconds


array([[-9.8158304e-10],
       [-1.2277140e-09],
       [-1.1826810e-07],
       [-1.4780385e-07]], dtype=float32)

ต่อมาคือการใช้ JAX คำนวณฟังก์ชันที่เป็นการหาอนุพันธ์อันดับสอง คือ dAdx(), dBdx(), dAdu(), 
dBdu() โดยมีการจับเวลาที่ใช้ในการคำนวณ

In [62]:
ad_pkg = 1  # select JAX AD
x_star = jnp.array([0.0, torch.pi, 0.0, 0.0]).reshape(4,1)
u_star = 1.0

In [63]:
# dAdx (JAX)
start_time = time.perf_counter()
Ax_jax = jnp.squeeze(dAdx(x_star,u_star))
end_time = time.perf_counter()
jax_dadx_time = end_time - start_time
print(f"Elapsed time (JAX : dAdx) : {jax_dadx_time:.6f} seconds")
Ax_jax

Elapsed time (JAX : dAdx) : 2.575635 seconds


Array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -2.14138301e-03,  0.00000000e+00,
        -6.37206031e-05],
       [ 0.00000000e+00, -3.19725974e-03,  0.00000000e+00,
        -7.95387823e-05],
       [ 0.00000000e+00, -1.05343044e-01,  0.00000000e+00,
        -4.30992106e-03],
       [ 0.00000000e+00, -1.57050446e-01,  0.00000000e+00,
        -5.38383238e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
      

In [64]:
# dBdx (JAX)
start_time = time.perf_counter()
Bx_jax = dBdx(x_star,u_star)
end_time = time.perf_counter()
jax_dbdx_time = end_time - start_time
print(f"Elapsed time (JAX : dBdx) : {jax_dbdx_time:.6f} seconds")
Bx_jax

Elapsed time (JAX : dBdx) : 1.897491 seconds


Array([[ 0.0000000e+00, -1.3262699e-06,  0.0000000e+00, -4.5935423e-08],
       [ 0.0000000e+00, -1.5931810e-06,  0.0000000e+00, -5.2526463e-08],
       [ 0.0000000e+00, -1.1780267e-04,  0.0000000e+00, -4.4945295e-06],
       [ 0.0000000e+00, -1.4988930e-04,  0.0000000e+00, -5.2902114e-06]],      dtype=float32)

In [65]:
# dAdu (JAX)
start_time = time.perf_counter()
Au_jax = dAdu(x_star,u_star)
end_time = time.perf_counter()
jax_dadu_time = end_time - start_time
print(f"Elapsed time (JAX : dAdu) : {jax_dadu_time:.6f} seconds")
Au_jax

Elapsed time (JAX : dAdu) : 2.065408 seconds


Array([[ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [-1.3262699e-06],
       [-1.5931810e-06],
       [-1.1780267e-04],
       [-1.4988930e-04],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [-4.5935423e-08],
       [-5.2526467e-08],
       [-4.4945295e-06],
       [-5.2902114e-06]], dtype=float32)

In [66]:
# dBdu (JAX)
start_time = time.perf_counter()
Bu_jax = dBdu(x_star,u_star)
end_time = time.perf_counter()
jax_dbdu_time = end_time - start_time
print(f"Elapsed time (JAX : dBdu) : {jax_dbdu_time:.6f} seconds")
Bu_jax

Elapsed time (JAX : dBdu) : 1.423196 seconds


Array([[-9.8158304e-10],
       [-1.2277140e-09],
       [-1.1826810e-07],
       [-1.4780385e-07]], dtype=float32)

วัดความแตกต่างโดยนอร์มของเมทริกซ์ที่ได้จากการคำนวณโดย torch และ JAX

In [67]:
from numpy.linalg import norm

In [69]:
dAx = norm(Ax_torch - Ax_jax)
dBx = norm(Bx_torch - Bx_jax)
dAu = norm(Au_torch - Au_jax)
dBu = norm(Bu_torch - Bu_jax)

สรุปเวลาในการคำนวณ และความแตกต่างระหว่างการใช้ torch และ JAX

In [78]:
print("AD (2nd order)    torch (sec)       JAX (sec)     difference between results (norm)")
print("    dAdx            {}            {}          {}".format(
    round(torch_dadx_time,3),round(jax_dadx_time,3),dAx))
print("    dBdx            {}            {}          {}".format(
    round(torch_dbdx_time,3),round(jax_dbdx_time,3),dBx))
print("    dAdu            {}            {}          {}".format(
    round(torch_dadu_time,3),round(jax_dadu_time,3),dAu))
print("    dBdu            {}            {}          {}".format(
    round(torch_dbdu_time,3),round(jax_dbdu_time,3),dBu))

AD (2nd order)    torch (sec)       JAX (sec)     difference between results (norm)
    dAdx            0.049            2.576          0.0
    dBdx            0.069            1.897          7.290155458472558e-12
    dAdu            0.071            2.065          4.547612286742719e-13
    dBdu            0.069            1.423          0.0


จะเห็นว่าผลลัพธ์ที่ได้จากการคำนวณโดย torch และ JAX มีความแตกต่างกันน้อยมาก 
(วัดโดยนอร์มของความแตกต่างระหว่างเมทริกซ์) แต่สำหรับเวลาที่ใช้ในการคำนวณ torch 
มีสมรรถนะเหนือกว่า JAX อย่างเด่นชัด นี่เป็นเหตุผลสำคัญที่เราควรเลือกใช้ torch autograd 
ในอัลกอริทึมที่มีการวนซ้ำหลายรอบ เช่น DDP/iLQR ในบทที่ 5

## บรรณานุกรม

1. PyTorch website : https://pytorch.org/ 

2. Z. Manchester et.al. [16-745 Optimal Control & Reinforcement Learning, 
Course materials](https://optimalcontrol.ri.cmu.edu/#learning-resources), Carnegie Mellon University. 2025.

3. R. Tedrake. [Underactuated Robotics: Algorithms for Walking, Running, Swimming, Flying, and Manipulation (Course Notes for MIT 6.832)](https://underactuated.csail.mit.edu). 2023. 



<div align="center">
<img src="https://raw.githubusercontent.com/dewdotninja/sharing-github/refs/heads/master/dewninja_logo50.jpg" alt="dewninja"/>
</div>
<div align="center">dew.ninja 2025</div>